In [1]:
# Install folium for creating interactive maps
!pip install folium tenacity


In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import folium
from folium.plugins import MarkerCluster
from tenacity import retry, stop_after_attempt, wait_fixed

# Load environment variables from .env file
env_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/API Keys/.env'
load_dotenv(env_path)

# Get the API key from the environment variables
api_key = os.getenv('API_KEY')

# Check if the API key was successfully loaded
if not api_key:
    raise ValueError("Error: The API key was not found in the .env file.")



In [3]:
# Function to get Whole Foods locations using the Google Places API with retry mechanism
@retry(stop=stop_after_attempt(5), wait=wait_fixed(2))
def get_whole_foods_locations(api_key, location, radius=50000, keyword='Whole Foods Market'):
    places = []
    next_page_token = None

    while True:
        # Construct the API request URL
        url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={keyword}&location={location}&radius={radius}&key={api_key}"
        if next_page_token:
            url += f"&pagetoken={next_page_token}"
        
        response = requests.get(url)  # Make the API request
        if response.status_code == 200:
            data = response.json()  # Parse the JSON response
            places.extend(data['results'])  # Add results to the list
            next_page_token = data.get('next_page_token')  # Get the next page token
            if not next_page_token:
                break
        else:
            print("Error:", response.status_code)  # Print error if request fails
            break
    return places

# Function to geocode an address (not used in the main flow but included for completeness)
def geocode_address(api_key, address):
    # Construct the API request URL
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)  # Make the API request
    if response.status_code == 200:
        location = response.json()['results'][0]['geometry']['location']  # Extract the location data
        return location['lat'], location['lng']
    else:
        return None, None


In [4]:
import numpy as np

# Function to generate a grid of lat/lng points across the US
def generate_grid(min_lat, max_lat, min_lng, max_lng, step_lat, step_lng):
    lat_points = np.arange(min_lat, max_lat, step_lat)
    lng_points = np.arange(min_lng, max_lng, step_lng)
    grid = [(lat, lng) for lat in lat_points for lng in lng_points]
    return grid

# Define the bounds for the United States
min_lat, max_lat = 24.396308, 49.384358  # Latitude bounds
min_lng, max_lng = -125.0, -66.93457    # Longitude bounds
step_lat, step_lng = 1.0, 1.0           # Step sizes for the grid

# Generate the grid of center points
locations = generate_grid(min_lat, max_lat, min_lng, max_lng, step_lat, step_lng)
locations_str = [f"{lat},{lng}" for lat, lng in locations]

# Output the number of locations to ensure the grid is generated correctly
print(f"Number of locations: {len(locations_str)}")


Number of locations: 1475


In [5]:
# Fetch Whole Foods locations from all grid points
all_places = []
for loc in locations_str:
    try:
        places = get_whole_foods_locations(api_key, location=loc)
        all_places.extend(places)
        print(f"Fetched data for location: {loc}")  # Print progress
    except Exception as e:
        print(f"Failed to fetch data for location {loc}: {e}")

# Output the number of places fetched
print(f"Total places fetched: {len(all_places)}")


Fetched data for location: 24.396308,-125.0
Fetched data for location: 24.396308,-124.0
Fetched data for location: 24.396308,-123.0
Fetched data for location: 24.396308,-122.0
Fetched data for location: 24.396308,-121.0
Fetched data for location: 24.396308,-120.0
Fetched data for location: 24.396308,-119.0
Fetched data for location: 24.396308,-118.0
Fetched data for location: 24.396308,-117.0
Fetched data for location: 24.396308,-116.0
Fetched data for location: 24.396308,-115.0
Fetched data for location: 24.396308,-114.0
Fetched data for location: 24.396308,-113.0
Fetched data for location: 24.396308,-112.0
Fetched data for location: 24.396308,-111.0
Fetched data for location: 24.396308,-110.0
Fetched data for location: 24.396308,-109.0
Fetched data for location: 24.396308,-108.0
Fetched data for location: 24.396308,-107.0
Fetched data for location: 24.396308,-106.0
Fetched data for location: 24.396308,-105.0
Fetched data for location: 24.396308,-104.0
Fetched data for location: 24.39

In [9]:
# Remove duplicates based on place_id
unique_places = {place['place_id']: place for place in all_places}.values()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(unique_places)

# Flatten the nested 'geometry' column to get latitude and longitude
df['latitude'] = df['geometry'].apply(lambda x: x['location']['lat'])
df['longitude'] = df['geometry'].apply(lambda x: x['location']['lng'])

# Select relevant columns and rename them for clarity
df = df[['name', 'formatted_address', 'rating', 'user_ratings_total', 'latitude', 'longitude']]

# Display the first few rows of the DataFrame
display(df.head())

df.count()

,name,formatted_address,rating,user_ratings_total,latitude,longitude
0,Whole Foods Market,"6350 W 3rd St, Los Angeles, CA 90036, United S...",4.3,1873,34.070308,-118.360713
1,Whole Foods Market,"2520 Glendale Blvd, Los Angeles, CA 90039, Uni...",4.2,1437,34.102881,-118.258434
2,Whole Foods Market,"788 S Grand Ave, Los Angeles, CA 90017, United...",4.4,3934,34.046117,-118.257480
3,Whole Foods Market,"7881 Edinger Ave #150, Huntington Beach, CA 92...",4.2,1126,33.731116,-117.990553
4,Whole Foods Market,"2847 Park Ave, Tustin, CA 92782, United States",4.3,1412,33.696213,-117.825457


name                  2588
formatted_address     2588
rating                2588
user_ratings_total    2588
latitude              2588
longitude             2588
dtype: int64

In [10]:
# Save the DataFrame to a CSV file
df.to_csv('whole_foods_locations.csv', index=False)
print("Whole Foods locations have been saved to 'whole_foods_locations.csv'")


Whole Foods locations have been saved to 'whole_foods_locations.csv'


In [13]:
# Path to the input CSV file
input_csv_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/whole_foods_locations.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_csv_path)

# Filter the DataFrame to include only rows where the "name" column is "Whole Foods Market"
filtered_df = df[df['name'] == 'Whole Foods Market']

# Path to save the filtered CSV file
output_csv_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Whole_Foods_Market_Locations.csv'

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv(output_csv_path, index=False)

print(f"Filtered data has been saved to '{output_csv_path}'")


Filtered data has been saved to '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Whole_Foods_Market_Locations.csv'


In [24]:
# Path to the filtered CSV file
filtered_csv_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Whole_Foods_Market_Locations.csv'

# Read the filtered CSV file into a DataFrame
filtered_df = pd.read_csv(filtered_csv_path)

# Display the DataFrame
print("Filtered DataFrame:")
print(filtered_df)

# Count the values in the DataFrame
value_counts = filtered_df['name'].value_counts()

# Display the value counts
print("\nValue Counts:")
print(value_counts)



Filtered DataFrame:
                   name                                  formatted_address  \
0    Whole Foods Market  6350 W 3rd St, Los Angeles, CA 90036, United S...   
1    Whole Foods Market  2520 Glendale Blvd, Los Angeles, CA 90039, Uni...   
2    Whole Foods Market  788 S Grand Ave, Los Angeles, CA 90017, United...   
3    Whole Foods Market  7881 Edinger Ave #150, Huntington Beach, CA 92...   
4    Whole Foods Market     2847 Park Ave, Tustin, CA 92782, United States   
..                  ...                                                ...   
521  Whole Foods Market            951 Bank St, Ottawa, ON K1S 3W7, Canada   
522  Whole Foods Market  4755 Fauntleroy Wy SW Ste 190, Seattle, WA 981...   
523  Whole Foods Market  120 13th St E, North Vancouver, BC V7L 4W8, Ca...   
524  Whole Foods Market  1030 Lakeway Dr, Bellingham, WA 98229, United ...   
525  Whole Foods Market    925 Main St, West Vancouver, BC V7T 2Z3, Canada   

     rating  user_ratings_total   latitude 

In [25]:
# Path to the filtered CSV file
filtered_csv_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Whole_Foods_Market_Locations.csv'

# Read the filtered CSV file into a DataFrame
filtered_df = pd.read_csv(filtered_csv_path)

# Display the DataFrame
print("Filtered DataFrame:")
print(filtered_df)

# Count the values in the DataFrame
value_counts = filtered_df['name'].value_counts()

# Display the value counts
print("\nValue Counts:")
print(value_counts)

# Create a map centered around the geographical center of the US
map_center = [39.8283, -98.5795]  # Latitude and Longitude of the geographic center of the contiguous United States
m = folium.Map(location=map_center, zoom_start=5)

# Add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(m)

# Add markers to the map
for idx, row in filtered_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['formatted_address'],
        tooltip=row['name']
    ).add_to(marker_cluster)

# Display the map in Jupyter Notebook
m

# Save the map to an HTML file
map_output_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Whole_Foods_Market_Map.html'
m.save(map_output_path)

print(f"Interactive map has been saved to '{map_output_path}'")


Filtered DataFrame:
                   name                                  formatted_address  \
0    Whole Foods Market  6350 W 3rd St, Los Angeles, CA 90036, United S...   
1    Whole Foods Market  2520 Glendale Blvd, Los Angeles, CA 90039, Uni...   
2    Whole Foods Market  788 S Grand Ave, Los Angeles, CA 90017, United...   
3    Whole Foods Market  7881 Edinger Ave #150, Huntington Beach, CA 92...   
4    Whole Foods Market     2847 Park Ave, Tustin, CA 92782, United States   
..                  ...                                                ...   
521  Whole Foods Market            951 Bank St, Ottawa, ON K1S 3W7, Canada   
522  Whole Foods Market  4755 Fauntleroy Wy SW Ste 190, Seattle, WA 981...   
523  Whole Foods Market  120 13th St E, North Vancouver, BC V7L 4W8, Ca...   
524  Whole Foods Market  1030 Lakeway Dr, Bellingham, WA 98229, United ...   
525  Whole Foods Market    925 Main St, West Vancouver, BC V7T 2Z3, Canada   

     rating  user_ratings_total   latitude 